# Manuelna anliza Log fajla kroz Python


### Ucitavanje log fajl

In [11]:

log_file_path = './log2.logs'

with open(log_file_path, 'r') as file:
    logs = file.readlines()


### izdvajanje  IP adrese 

In [2]:
import re

# regex patern za izdvajanje ip adrese iz fajla
ip_regex = r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b'


ip_addresses = set()

for log in logs:
    ip_match = re.search(ip_regex, log)
    if ip_match:
        ip_addresses.add(ip_match.group())


print("IP adrese")
print(ip_addresses)


IP adrese
{'154.144.187.133', '248.46.133.115', '183.85.218.115', '201.226.67.16', '199.54.201.208', '12.78.81.208', '255.139.131.54', '137.81.144.137', '124.120.36.214', '198.14.11.206', '211.172.102.34', '175.168.53.120', '144.54.99.146', '11.168.221.20', '40.76.105.193', '22.154.98.70', '51.130.43.213', '255.57.117.122', '88.88.76.224', '88.58.170.102', '170.172.140.118', '218.149.94.105', '1.2.29.84', '38.249.24.144', '243.83.177.179', '16.57.146.59', '57.92.217.14', '159.153.5.199', '9.77.150.91', '132.121.50.217', '35.16.32.234', '118.135.239.119', '53.168.131.176', '33.197.181.39', '70.191.57.38', '225.217.175.75', '198.7.122.231', '115.163.130.59', '6.27.233.69', '53.70.254.119', '113.154.244.177', '56.250.76.28', '214.254.208.9', '120.106.233.133', '41.240.211.153', '184.176.184.11', '107.176.162.238', '19.224.220.209', '192.218.24.53', '23.72.32.210', '30.146.182.16', '186.126.144.22', '250.192.103.76', '22.141.65.180', '123.178.221.34', '232.11.227.113', '134.64.62.85', '31.

### Izvlacenje logova koji sadrže indikatore napada

In [3]:
# Definiši ključne fraze koje ukazuju na napad
attack_indicators = ['SELECT', 'UNION', 'DROP', 'alert(', '<script>', '--', "' OR '1'='1"]

suspicious_logs = []

for log in logs:
    if any(indicator in log for indicator in attack_indicators):
        suspicious_logs.append(log)

print("Logovi sa indikacijama napada:")
print(suspicious_logs)


Logovi sa indikacijama napada:
[]


### Grupisanje logova po IP adresama i broju zahteva radi pokusaja detekcije DDoS napada
Najpre se  vrsi grupisati logove prema IP adresama i izračunati broj zahteva po svakoj IP adresi:

In [4]:
from collections import defaultdict

# Broj zahteva po IP adresi
ip_request_count = defaultdict(int)

for log in logs:
    ip_match = re.search(ip_regex, log)
    if ip_match:
        ip_request_count[ip_match.group()] += 1

# Ispis IP adresa sa brojem zahteva
for ip, count in ip_request_count.items():
    print(f"IP adresa: {ip}, Broj zahteva: {count}")


IP adresa: 164.25.136.21, Broj zahteva: 1
IP adresa: 152.218.149.119, Broj zahteva: 1
IP adresa: 159.252.247.116, Broj zahteva: 1
IP adresa: 230.249.203.159, Broj zahteva: 1
IP adresa: 164.142.49.216, Broj zahteva: 1
IP adresa: 27.79.158.34, Broj zahteva: 1
IP adresa: 88.193.11.230, Broj zahteva: 1
IP adresa: 253.92.115.78, Broj zahteva: 1
IP adresa: 118.17.253.80, Broj zahteva: 1
IP adresa: 238.79.181.126, Broj zahteva: 1
IP adresa: 104.182.136.92, Broj zahteva: 1
IP adresa: 2.208.150.246, Broj zahteva: 1
IP adresa: 74.183.8.133, Broj zahteva: 1
IP adresa: 25.183.10.138, Broj zahteva: 1
IP adresa: 175.243.141.187, Broj zahteva: 1
IP adresa: 216.182.98.142, Broj zahteva: 1
IP adresa: 76.78.177.31, Broj zahteva: 1
IP adresa: 84.30.4.230, Broj zahteva: 1
IP adresa: 206.95.176.208, Broj zahteva: 1
IP adresa: 79.197.247.25, Broj zahteva: 1
IP adresa: 83.216.128.52, Broj zahteva: 1
IP adresa: 139.240.150.205, Broj zahteva: 1
IP adresa: 135.248.69.54, Broj zahteva: 1
IP adresa: 158.88.66.211

### Prepoznavanje DDoS napada
Definišemo prag za broj zahteva sa iste IP adrese koji smatramo sumnjivim za DDoS napad. Na primer, ako jedna IP adresa šalje više od 100 zahteva u vrlo kratkom vremenskom intervalu, možemo je smatrati potencijalnim napadačem.

In [5]:

ddos_threshold = 100

potential_ddos_ips = {ip: count for ip, count in ip_request_count.items() if count > ddos_threshold}


print("IP adrese sumnjive na DDoS napad:")
for ip, count in potential_ddos_ips.items():
    print(f"IP adresa: {ip}, Broj zahteva: {count}")


IP adrese sumnjive na DDoS napad:


### Brute-force napadi
Identifikovanje brute-force napade tako što tražimo više neuspešnih pokušaja prijave sa iste IP adrese. Ako logovi sadrže fraze poput "login failed", možemo koristiti ovaj obrazac.

In [6]:
# Detekcija directory traversal napada
traversal_pattern = r'\.\./'
traversal_logs = []

for log in logs:
    if re.search(traversal_pattern, log):
        traversal_logs.append(log)

# Pronađi IP adrese povezane sa directory traversal napadima
traversal_ips = set()

for log in traversal_logs:
    ip_match = re.search(ip_regex, log)
    if ip_match:
        traversal_ips.add(ip_match.group())

print("IP adrese povezane sa directory traversal napadima:")
print(traversal_ips)


IP adrese povezane sa directory traversal napadima:
set()


### Directory Traversal napadi
Directory traversal napadi pokušavaju pristupiti fajlovima van dozvoljene strukture direktorijuma, koristeći obrasce kao što su ../.

In [7]:
# Detekcija directory traversal napada
traversal_pattern = r'\.\./'
traversal_logs = []

for log in logs:
    if re.search(traversal_pattern, log):
        traversal_logs.append(log)

# Pronađi IP adrese povezane sa directory traversal napadima
traversal_ips = set()

for log in traversal_logs:
    ip_match = re.search(ip_regex, log)
    if ip_match:
        traversal_ips.add(ip_match.group())

print("IP adrese povezane sa directory traversal napadima:")
print(traversal_ips)


IP adrese povezane sa directory traversal napadima:
set()


### Command Injection napadi
Command injection napadi koriste specifične simbole kao što su &&, |, ; kako bi izvršili neovlašćene komande.

In [8]:

command_injection_patterns = [
    r'(?:;|&&|\|)\s*(?:cat|ls|whoami|uname|pwd)',  
    r'(?:;|&&|\|)\s*(curl|wget)', 
    r'[$`]',  
    r'(>.+)',  
]

command_injection_logs = []

# Detekcija command injection napada
for log in logs:
    if any(re.search(pattern, log) for pattern in command_injection_patterns):
        command_injection_logs.append(log)

# Ispis logova koji sadrže command injection napade
print("Logovi sa command injection napadima:")
for log in command_injection_logs:
    print(log)


Logovi sa command injection napadima:


### Remote File Inclusion (RFI)
Napadi sa uključenjem udaljenih fajlova koriste URL-ove u ulaznim parametrima kako bi preuzeli zlonamerne fajlove na server.

In [9]:
# Detekcija RFI napada
rfi_pattern = r'http[s]?://'
rfi_logs = []

for log in logs:
    if re.search(rfi_pattern, log):
        rfi_logs.append(log)

# Pronađi IP adrese povezane sa RFI napadima
rfi_ips = set()

for log in rfi_logs:
    ip_match = re.search(ip_regex, log)
    if ip_match:
        rfi_ips.add(ip_match.group())

print("IP adrese povezane sa RFI napadima:")
print(rfi_ips)


IP adrese povezane sa RFI napadima:
set()


In [18]:
import re

# Primer logova (u stvarnosti, ovi logovi dolaze iz servera ili aplikacije)
# logs = [
#     'GET /?url=http://localhost/admin HTTP/1.1',  # SSRF
#     'POST /transfer HTTP/1.1',  # CSRF bez tokena
#     '<script>alert(1)</script>',  # XSS
#     'uname -a; ls',  # Command Injection
# ]

# Regex obrasci za različite napade

sqlInjection = [
    r'(\%27|\%22|\'|\"|;|\-\-|\%23|\#).*?\b(select|union|insert|update|delete|drop|from)\b'
]

command_injection_patterns = [
    r'(?:;|&&|\|)\s*(?:cat|ls|whoami|uname|pwd)',  
    r'(?:;|&&|\|)\s*(curl|wget)', 
    r'[$`]',  
    r'(>.+)',  
]

ssrf_patterns = [
    r'http[s]?://(?:localhost|127\.0\.0\.1|0\.0\.0\.0|169\.254|internal)',
    r'http[s]?://.*\b(onion|admin|metadata)\b',
]

csrf_patterns = [
    r'POST\s+.*\s+HTTP/1\.1',  
    r'(?!csrf_token)',
]

xss_patterns = [
    r'<script.*?>.*?</script>',  
    r'javascript:',  
    r'on\w+\s*=',
    r'&#\d+;',  
]

# Funkcija za pretragu logova po zadatim obrascima
def search_logs(logs, patterns):
    matched_logs = []
    for log in logs:
        if any(re.search(pattern, log) for pattern in patterns):
            matched_logs.append(log)
    return matched_logs

# Detekcija napada u logovima
command_injection_logs = search_logs(logs, command_injection_patterns)
ssrf_logs = search_logs(logs, ssrf_patterns)
csrf_logs = search_logs(logs, csrf_patterns)
xss_logs = search_logs(logs, xss_patterns)
sql_logs = search_logs(logs,sqlInjection)

# Ispis rezultata
print("Logovi sa command injection napadima:")
for log in command_injection_logs:
    print(log)

print("\nLogovi sa SSRF napadima:")
for log in ssrf_logs:
    print(log)

print("\nLogovi sa CSRF napadima:")
for log in csrf_logs:
    print(log)

print("\nLogovi sa XSS napadima:")
for log in xss_logs:
    print(log)

print("\n SQL injection")
for log in sql_logs:
    print(log)


Logovi sa command injection napadima:

Logovi sa SSRF napadima:

Logovi sa CSRF napadima:
164.25.136.21 - - [10/Sep/2024:09:39:11 +0200] "GET /local%20area%20network/encoding.php?id=12345 HTTP/1.1" 200 2634 "-" "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/5342 (KHTML, like Gecko) Chrome/39.0.892.0 Mobile Safari/5342"

152.218.149.119 - - [10/Sep/2024:09:39:11 +0200] "GET /task-force/initiative/conglomeration.php HTTP/1.1" 200 3039 "-" "Opera/9.69 (X11; Linux i686; en-US) Presto/2.13.210 Version/11.00"

159.252.247.116 - - [10/Sep/2024:09:39:11 +0200] "GET /moderator-Compatible.js?search=info HTTP/1.1" 302 54 "-" "Mozilla/5.0 (X11; Linux i686) AppleWebKit/5310 (KHTML, like Gecko) Chrome/38.0.857.0 Mobile Safari/5310"

230.249.203.159 - - [10/Sep/2024:09:39:11 +0200] "GET /focus%20group-systemic.php?name=settings HTTP/1.1" 200 1060 "-" "Mozilla/5.0 (Windows NT 6.0) AppleWebKit/5352 (KHTML, like Gecko) Chrome/40.0.815.0 Mobile Safari/5352"

164.142.49.216 - - [10/Sep/2024:09:39:11 +0200] "GE

### Base 64

In [ ]:
import re

# Regex za prepoznavanje Base64 stringa
base64_pattern = r'(?:[A-Za-z0-9+/]{4}){2,}(?:[A-Za-z0-9+/]{2}==|[A-Za-z0-9+/]{3}=)?'

# Primeri URL-ova sa Base64 enkodovanim stringovima
# urls = [
#     'https://example.com?data=SGVsbG8gd29ybGQ=',  # Base64: Hello world
#     'https://example.com?query=U29tZSBvdGhlciBkYXRh',  # Base64: Some other data
#     'https://example.com?data=HelloWorld',  # Regular string (nije Base64)
#     'https://example.com?token=dGVzdA==',  # Base64: test
#     'https://example.com',  # Nema Base64
# ]

# Funkcija za prepoznavanje Base64 stringova u URL-ovima
def find_base64_in_urls(urls):
    for url in urls:
        base64_strings = re.findall(base64_pattern, url)
        if base64_strings:
            print(f"URL: {url}")
            for base64_string in base64_strings:
                print(f"  Pronađen Base64 enkodovan string: {base64_string}")

# Pretraži URL-ove za Base64 enkodovane stringove
find_base64_in_urls(urls)


### Dodatak

In [ ]:
import re

# Definišemo regex za prepoznavanje Base64 stringova i XSS napada
#base64_pattern = r'(?:[A-Za-z0-9+/]{4}){2,}(?:[A-Za-z0-9+/]{2}==|[A-Za-z0-9+/]{3}=)?'
xss_pattern = r'<script.*?>|%3Cscript.*?%3E'  # XSS obrasci (plain text ili URL encoded)
sql_injection_pattern = r'(\%27|\%22|\'|\"|;|\-\-|\%23|\#).*?\b(select|union|insert|update|delete|drop|from)\b'
command_injection_pattern = r'(;|\|\||&&).*?(cat|ls|whoami|pwd|uname)'


# Funkcija za pretragu potencijalnih napada
def analyze_logs(logs):
    for log in logs:
        # Proveri da li postoji Base64 enkodovani string
        
        
        # Proveri na XSS napad
        if re.search(xss_pattern, log, re.IGNORECASE):
            print(f"XSS napad otkriven u logu: {log}")
        
        # Proveri na SQL Injection
        if re.search(sql_injection_pattern, log, re.IGNORECASE):
            print(f"SQL Injection otkriven u logu: {log}")
        
        # Proveri na Command Injection
        if re.search(command_injection_pattern, log, re.IGNORECASE):
            print(f"Command Injection otkriven u logu: {log}")

# Pozovi funkciju za analizu logova
analyze_logs(logs)


### First   

In [34]:
import re
from collections import Counter

# Otvori log fajl
with open('log2.logs', 'r') as log_file:
    logs = log_file.readlines()

# Definiši unapređene regex paterne za različite tipove napada
patterns = {
    'SQL Injection': re.compile(
        r"(?i)(\%27|'|--|;|/\*|\*\/|#|\%23)(.*?(SELECT|INSERT|UPDATE|DELETE|DROP|UNION|AND|OR).*)"
    ),
    'XSS': re.compile(r"(<|%3C)(script|img|iframe|on\w+)(>|%3E)"),
    'Command Injection': re.compile(r"(\&\&|\|\||\$(\(|\{))"),
    'SSRF': re.compile(
        r"(?i)\b(GET|POST)\s+.*(url|dest|redirect|uri|path)=((https?:\/\/)?(localhost|127\.0\.0\.1|169\.254\.169\.254|192\.168\.\d{1,3}\.\d{1,3}|10\.\d{1,3}\.\d{1,3}\.\d{1,3}|172\.(1[6-9]|2[0-9]|3[0-1])\.\d{1,3}))"
    ),
    'CSRF': re.compile(r"(POST|GET) [^\n]*\?token=[^\n]*"),
   
}

# Brojač za detekciju DDoS napada
ip_addresses = []

# Proveri svaki log za potencijalne napade
for log in logs:
    # Ekstraktuj IP adresu
    ip_match = re.match(r"(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})", log)
    if ip_match:
        ip_addresses.append(ip_match.group(1))
    
    # Proveri za druge napade
    for attack_type, pattern in patterns.items():
        if pattern.search(log):
            print(f"[!] Mogući {attack_type} napad detektovan:\n{log}")

# Detekcija potencijalnog DDoS-a na osnovu IP adrese
ip_count = Counter(ip_addresses)
for ip, count in ip_count.items():
    if count > 10:  # Prag: više od 10 zahteva u kratkom periodu
        print(f"[!] Mogući DDoS napad detektovan sa IP adrese: {ip}, broj zahteva: {count}")


[!] Mogući SQL Injection napad detektovan:
192.168.1.100 - - [10/Sep/2024:10:15:21 +0200] "GET /product.php?id=12345' OR '1'='1'-- HTTP/1.1" 200 1500 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"

[!] Mogući XSS napad detektovan:
192.168.1.105 - - [10/Sep/2024:11:30:45 +0200] "GET /search.php?query=<script>alert('XSS');</script> HTTP/1.1" 200 1300 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"

[!] Mogući SSRF napad detektovan:
192.168.1.100 - - [10/Sep/2024:12:30:10 +0200] "GET /api/v1/resource?url=http://localhost:8080/admin HTTP/1.1" 200 1500 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"

